# Data Science Bowl 2019

This script contains the code used to generate my 204th place solution to the 2019 data science bowl.  Some sections of the code are taken from public notebooks - this is acknowledged for these cell blocks but no links are provided since I am writing this some time after the competition ended.  The purpose of this write-up is as a summary of the lessons learned throughout this competition. 

## Objective

The objective is to classify the performance of users of the PBS kids app using information about the user's activities.  Each user has a unique identification code.  The performance is measured by the number of attempts a user took before successfully completed an assessment.  There are four classes:

*  3 - The user was successful on their first attempt.
*  2 - The user was successful on their second attempt.
*  1 - The user successfully completed the assessment task on any subsequent attempt.
*  0 - The user attempted but did not successfully complete the assessment.

The predictions are scored using the quadratic weighted kappa metric, which is a measurement of agreement between two outcomes.  This metric penalises predictions that are further from the actual class - this is important for this competition since each category is not incomparable to the others.  Intuitively, for a ground truth class of 3, a model predicting 2 should be better than a model predicting 0.

## Model

The model chosen for this task is a lightgbm gradient boosted decision tree.  This model was chosen for it's predictive accuracy and fast training.  5 group folds were used for cross-validation, with the groups being split by user identification code.  This means that each validation set contains identification codes not present in the training set.  This validation method was chosen as it emulates the test data, which contains many user identification codes not present in the training set.  

Though this is a classification task, a regression model was used with cutoff points used to generate class predictions.  The cutoff points are chosen uses a minimiser imported from the scipy module.  The prediction process is best explained using an example - say the model predicts 1.4 for the user's performance on some assessment.  The cutoff points chosen by the minimiser for the class with the label 1 are (0.7, 1.5).  The prediction lies in this interval, so the class predicted by the model is 1.  This prediction method had a better performance as measured by the group-fold cross-validation and the public leaderboard.  The probably reason for this increase in performace is because the quadratic weighted kappa metric does not penalise wrong predictions equally for each class - the classification objective does not capture this effect.

## Features

The contents of the data are explained on the competition page.  Hundreds of features were extracted for use in the model prior to any feature selection.  Typical features extracted were past performance on assessments, assessment type, games played in the app and summary statistics for the event codes.  The meaning of particular event codes are explained in the 'specs' file - the more useful codes were usually the codes indicating a game has started or finished, or a particular level was reached inside an activity.  The summary statistics were usually either the sum or the mean number for a particular event code.

The most consistently useful feature was a user's past performance on any type of assessment.  Almost every other feature had a very small impact on the predictive accuracy.  This was a clear challenge for the task - there are hundreds of available features and almost all of them had very little impact on the performance as measured by the quadratic weighted kappa.  To address this, a sort of backward stepwise selection was used - the full model was scored using the cross-validation method described above.  For each step, a random selection of 10 percent of the total features were removed and a new model was trained and scored.  This method is intractable for every possible subset of features, so a fixed number of trials was used instead.  After these trials are completed, the best model as measured by the cross-validation method is chosen, and the process is repeated.  This is repeated for a fixed number of steps, resulting in a model with far fewer features used.  The first model used 898 features - a typical final model after this process would have about 500 features.  The number of steps was chosen based on the time taken to train each model - a typical model takes about 80 seconds to train including the group-fold validation, and the entire script should run in less then 9 hours. The total number of models trained should then be around 400 or less.  

## Ensembling

For this section, a model actually refers to the collection of 5 models trained using the group-fold validation method described in the model summary.

A holdout set was created for use in ensemble scoring from the test set.  The predictions are evaluated in this competition on the final assessment of each user in the test set.  I used the assessments prior to the final assessment for validation.

For each model trained, the score, the features used and the predictions on the holdout set are stored in a dictionary. The best 6 scoring models are then chosen as candidates for use in a simple linear regression model which predicts the class using a combination of only three of these model's predictions.  

The best subset of three models is chosen using the holdout predictions by first optimising the regression coefficients, then optimising the cutoff points used to transform regression predictions into class predictions.  Each subset is then scored using the combined out-of-fold predictions.  

The three best models are then used to generate test predictions which are combined using the linear regression coefficients and transformed to class predictions using the cutoff points optimised based on the quadratic weighted kappa mettric.

## Retrospective

This is my second competition on kaggle, the first being the ASHRAE Great Energy Predictor III.  The objective in the ASHRAE competition was to predict the meter readings for energy, steam, etc.  In the first competition, there were significant data leaks which were used by competitors as ensembling tools allowing very good scores on the public leaderboard.  The private dataset was not similar to the public dataset, and competitors such as myself who sought to use the leaks without a stable local validation method were punished.  I personally went from 240th on the public leaderboard to 799th on the private leaderboard.  I learned two main lessons from this competition - firstly, it is pointless to make modelling decisions such as feature selection or parameter tuning without a method that allows you to consistently determine whether an improvement has actually been made.  

I feel my attempts to address this issue in this competition were fruitful - my private leadboard score was correlated fairly strongly with both my local validation and the public leaderboard.  A particular problem with cross-validation for this competition was that the number of users who had no prior history on other assessments was much higher in the test set.  In earlier kernels I was using a sort of pseudo-bootstrapping technique to address this - samples were selected from the training set with replacement, with the class for each sample selected with probability corresponding to it's proportional appearance in the test set.  For example - about 40% of the test set samples had no prior assessment history.  The sample taken from the training set was chosen with a 40% probability of having no prior assessment history.  This process was repeated to generate many validation sets, and the scores were estimated using out-of-fold predictions.  This method had a higher correlation with the public leaderboard than the standard group-fold validation method.  The method was dropped in favour of a less stable, yet faster training method using only group-k-fold.  The justification was that the group-fold validation was adequate, and the improvement in training time was desirable despite a less stable cross-validation.

The second lesson learned from the ASHRAE competition was that cleaning a dataset is just as important as model tuning or ensembling.  The ASHRAE dataset was characterised by long streaks of zero readings for the various energy meters - all the winning models took significant steps to remove the entries corresponding to probable faulty meters.  This particular lesson was not as relevant for this competition since there was little evidence of bad data samples.

As for this competition - there were several problems in my approach which were evident upon looking at the winning submissions.  The most important, in my opinion, was my crude method of feature selection.  This was a particular issue for me in this competition - removing features that were measured to be non-influential by standard metrics available in lightgbm such as the gain importance and split importance did not lead to better validation scores.  My attempt to address this was to simply minimise the validation score by removing random subsets of up to 80 features at a time - frequently removing significant features.  I learned of permutation importance after reading the winning solution.  

Permutation importance works by creating a distribution of feature importances by randomly shuffling the class labels and recording the feature importances for each variable.  Repeating this allows an estimate of the mean importance of a particular variable when the labels are shuffled.  The mean of a sample is approximately normally distributed by the central limit theorem - this allows for a confidence interval to be assigned to the importance of each variable.  The importances of each variable when the labels are not shuffled can then be compared with these confidence intervals using p-values and standard hypothesis testing - if a variable has an importance which is considered too unlikely using some predetermined signficance level, it is probably useful for predicting the response variable.  This method was particularly effective here for several reasons; the training time is low enough to generate a reasonable null distribution, there are large numbers of variables which are highly correlated, most of which are not significant for predicting the correct class.  More information on this method is available here: 

https://academic.oup.com/bioinformatics/article/26/10/1340/193348

The second most important issue for me in this competition was my lack of version control.  At the end of the competition I have 147 versions of this code.  A frequent problem arises where I will want to reuse code I wrote 20 versions ago, but don't remember which version it was in.  I could also improve by writing down the ideas that I'm working on rather than relying on memory.

My focus going forward will be to improve version control, and to be more aware of state of the art methods.

In [ ]:
# Shortened the runtime since I only want the kernel for presentation purposes.  Set to true to run the original version.

run_long_version = False

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
import numpy as np
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')
import matplotlib.pylab as plt
import calendar
import scipy as sp
import warnings
warnings.filterwarnings("ignore")
np.random.seed(28)

import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from typing import Any
from numba import jit
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from itertools import product
import copy
import time
import json
import random

## Reading Data

In [ ]:
# Read in the csv files.  Removed the entries corresponding to the users who were active in the app but did not record and assessment results - they won't be very useful for the task.

root = '/kaggle/input/data-science-bowl-2019/'

train = pd.read_csv(root + 'train.csv')
train_labels = pd.read_csv(root + 'train_labels.csv')
specs = pd.read_csv(root + 'specs.csv')
test = pd.read_csv(root + 'test.csv')

train = train[train.installation_id.isin(train_labels.installation_id.unique())]
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")

## Feature Extraction

In [ ]:
# Dictionary recording the number of seconds for each clip.  Used in features recording time spent watching clips.

clip_time = {'Welcome to Lost Lagoon!':19,'Tree Top City - Level 1':17,'Ordering Spheres':61, 'Costume Box':61,
        '12 Monkeys':109,'Tree Top City - Level 2':25, 'Pirate\'s Tale':80, 'Treasure Map':156,'Tree Top City - Level 3':26,
        'Rulers':126, 'Magma Peak - Level 1':20, 'Slop Problem':60, 'Magma Peak - Level 2':22, 'Crystal Caves - Level 1':18,
        'Balancing Act':72, 'Lifting Heavy Things':118,'Crystal Caves - Level 2':24, 'Honey Cake':142, 'Crystal Caves - Level 3':19,
        'Heavy, Heavier, Heaviest':61}

In [ ]:
# Create dictionary of codes which have the description in each corresponding string.

descr = "how much time elapsed while the game was presenting feedback?"
feedback_ids = specs.loc[specs['info'].str.contains(descr)]['event_id'].values

descr2 = "how much time elapsed while the game was presenting instruction?"
instruction_ids = specs.loc[specs['info'].str.contains(descr2)]['event_id'].values

descr3 = "tutorial"
tutorial_ids = specs.loc[specs['info'].str.contains(descr3)]['event_id'].values

misses_ids = specs.loc[specs['args'].str.contains('misses')]['event_id'].values

id_list_dict = {'feedback_ids':feedback_ids, 'instruction_ids':instruction_ids, 'tutorial_ids':tutorial_ids, 'misses_ids':misses_ids}

In [ ]:
# Borrowed entirely from a public notebook.  

def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = (list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique())))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = (list(set(train['title'].unique()).union(set(test['title'].unique()))))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = (list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_event_id = (list(set(train['event_id'].unique()).union(set(test['event_id'].unique()))))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = (list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    train['hour'] = train['timestamp'].dt.hour
    test['hour'] = test['timestamp'].dt.hour
    train['weekday'] = train['timestamp'].dt.weekday
    test['weekday'] = test['timestamp'].dt.weekday
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

In [ ]:
# Mostly taken from a public notebook.  Added time duration features for different classes of activities.  Added features for the JSON data using the 
# dictionaries created from the specs file in the cell above.

def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0 
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    clip_durations = []
    Activity_durations = []
    Game_durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    
    game_event_code_count: Dict[str, int] = { str(ev) + '_g': 0 for ev in list_of_event_code}
    Activity_event_code_count: Dict[str, int] = {str(ev) + '_A': 0 for ev in list_of_event_code}    
    Activity_sum_event_count = 0
    game_sum_event_count = 0
    get_event_data_dict = {'feedback_ids':0, 'instruction_ids':0, 'tutorial_ids':0, 'misses_ids':0}
 #   event_id_duration_mean = {f'dur_mean_{idx}': 0 for idx in list_of_event_id}
 #   event_id_round = {f'round_{idx}': 0 for idx in list_of_event_id}
 #   event_id_level = {f'level_{idx}': 0 for idx in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    # itarates through each session of one instalation_id
    sessions_count = 0
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]     
        
        if session_type == 'Clip':
            clip_durations.append((clip_time[activities_labels[session_title]]))
        
        if session_type == 'Activity':
            Activity_durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = str(k) + '_A'
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            Activity_event_code_count = update_counters(Activity_event_code_count, "event_code")
        
        if session_type == 'Game':
            Game_durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = str(k) + '_g'
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            game_event_code_count = update_counters(game_event_code_count, "event_code")
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            features.update(get_event_data_dict)
            features.update()
            features['installation_session_count'] = sessions_count
            features['hour'] = session['hour'].iloc[-1]
            features['weekday'] = session['weekday'].iloc[-1]
            variety_features = [('var_event_code', event_code_count),
                              ('var_event_id', event_id_count),
                               ('var_title', title_count),
                               ('var_title_event_code', title_event_code_count)]
            
            for name, dict_counts in variety_features:
                arr = np.array(list(dict_counts.values()))
                features[name] = np.count_nonzero(arr)
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            features['game_sum_event_count'] = game_sum_event_count
            features['Activity_sum_event_count'] = game_sum_event_count
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
                features['duration_std'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                
            if clip_durations == []:
                features['Clip_duration_mean'] = 0
                features['Clip_duration_std'] = 0
            else:
                features['Clip_duration_mean'] = np.mean(clip_durations)
                features['Clip_duration_std'] = np.std(clip_durations)
                
            if Activity_durations == []:
                features['Activity_duration_mean'] = 0
                features['Activity_duration_std'] = 0
            else:
                features['Activity_duration_mean'] = np.mean(Activity_durations)
                features['Activity_duration_std'] = np.std(Activity_durations)
                
            if Game_durations == []:
                features['Game_duration_mean'] = 0
                features['Game_duration_std'] = 0
            else:
                features['Game_duration_mean'] = np.mean(Game_durations)
                features['Game_duration_std'] = np.std(Game_durations)
                
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        sessions_count += 1
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
        
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')
        
        # json data
        for id_list in get_event_data_dict.keys():
            if id_list == 'misses':
                just_ids = session.loc[session['event_id'].isin(id_list_dict[f'{id_list}'])]['event_data'].apply(lambda x:json.loads(x).get('misses', None)).sum()
            else:
                just_ids = session.loc[session['event_id'].isin(id_list_dict[f'{id_list}'])]['event_data'].apply(lambda x:json.loads(x).get('duration', None)).mean()
            if just_ids == just_ids:
                get_event_data_dict[f'{id_list}'] += just_ids
                get_event_data_dict[f'{id_list}'] /= 2
        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1], all_assessments[:-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [ ]:
# Call the utility function, from public notebook.
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)

In [ ]:
# Mostly from public notebook, modified slightly to include the dataframe "reduce_test_tr".  The original cell 
# discarded assessments in the test set prior to the final assessment - the only assessment used for evaluation in this competition.
# I chose to include assessments prior to the final assessment for validation purposes.

def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_test_tr = []
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 3614):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data, test_data_tr = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
        compiled_test_tr += (test_data_tr)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_test_tr = pd.DataFrame(compiled_test_tr)    
    categoricals = ['session_title']
    return reduce_train, reduce_test, reduce_test_tr, categoricals

In [ ]:
# Creates the datasets.

reduce_train, reduce_test, reduce_test_tr, categoricals = get_train_and_test(train, test)

In [ ]:
# From public notebook.  Only change is to include the reduce_test_tr dataframe.

def preprocess(reduce_train, reduce_test, reduce_test_tr):
    for df in [reduce_train, reduce_test, reduce_test_tr]:
        df['installation_session_clip_count'] = df.groupby(['installation_id'])['Clip'].transform('count')
        df['installation_session_game_count'] = df.groupby(['installation_id'])['Game'].transform('count')
        df['installation_session_act_count'] = df.groupby(['installation_id'])['Activity'].transform('count')
        df['installation_duration_mean'] = df.groupby(['installation_id'])['duration_mean'].transform('mean')
        df['installation_title_nunique'] = df.groupby(['installation_id'])['session_title'].transform('nunique')
        
        df['sum_event_code_count'] = df[[2050, 4100, 4230, 5000, 4235, 2060, 4110, 5010, 2070, 2075, 2080, 2081, 2083, 3110, 4010, 3120, 3121, 4020, 4021, 
                                        4022, 4025, 4030, 4031, 3010, 4035, 4040, 3020, 3021, 4045, 2000, 4050, 2010, 2020, 4070, 2025, 2030, 4080, 2035, 
                                        2040, 4090, 4220, 4095]].sum(axis = 1)
        
        df['installation_event_code_count_mean'] = df.groupby(['installation_id'])['sum_event_code_count'].transform('mean')
        
    features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
    features = [x for x in features if x not in ['accuracy_group', 'game_session']] + ['acc_' + title for title in assess_titles]
   
    return reduce_train, reduce_test, reduce_test_tr, features
# call feature engineering function
reduce_train, reduce_test, reduce_test_tr, features = preprocess(reduce_train, reduce_test, reduce_test_tr)

In [ ]:
# Mostly from public notebook.  Fixes bugs arising from using JSON strings for lightgbm feature names.

y = reduce_train['accuracy_group']
y_t = reduce_test_tr['accuracy_group']
reduce_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_train.columns]
reduce_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_test.columns]
reduce_test_tr.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_test_tr.columns]
features = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in features]

In [ ]:
# Functions used to calculate the cohen kappa metric, generate class predictions from regression predictions, optimise cutoff points.
# Some of these functions are from public kernels.  Others are from previous versions where I used the validation technique that attempted to emulate 
# the number of prior assessments.  

from functools import partial
from sklearn.base import BaseEstimator, TransformerMixin
@jit

def use_coefs(oof, coefs, train_preds=True):
    oof[oof <= coefs[0]] = 0
    oof[np.where(np.logical_and(oof > coefs[0], oof <= coefs[1]))] = 1
    oof[np.where(np.logical_and(oof > coefs[1], oof <= coefs[2]))] = 2
    oof[oof > coefs[2]] = 3
    if train_preds == True:
        return qwk(y, oof)
    else:
        return oof
    
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype='int')
    a2 = np.asarray(a2, dtype='int')

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

def obj_func(guess, y_pred, y_true):
    """
    Fast cappa eval function for lgb.
    """
    s = y_pred.copy()
    s[s <= guess[0]] = 0
    s[np.where(np.logical_and(s > guess[0], s <= guess[1]))] = 1
    s[np.where(np.logical_and(s > guess[1], s <= guess[2]))] = 2
    s[s > guess[2]] = 3
    score = -qwk(s, y_true)
    
    return score

def kappa_funcc(guess, y_pred, y_true):
    """
    Fast cappa eval function for lgb.
    """
    labels = y_true.get_label()
    y_pred[y_pred <= guess[0]] = 0
    y_pred[np.where(np.logical_and(y_pred > guess[0], y_pred <= guess[1]))] = 1
    y_pred[np.where(np.logical_and(y_pred > guess[1], y_pred <= guess[2]))] = 2
    y_pred[y_pred > guess[2]] = 3
    
    return 'cappa', qwk(y_pred, labels), True

def obs(x0, valid_ids, oof, y):
    scores = []
    for h in range(iters):
        scores.append(obj_func(coefs, oof[valid_ids[:,h]], y[valid_ids[:,h]]))
    return np.mean(scores)

## Feature selection

In [ ]:
# Create list of features.  Code removes duplicates created when fixing the broken JSON feature names so the dataset is usable in lightgbm. 

feat = [x for x in features if x not in ['installation_id']]
target = ['accuracy_group']
categoricals=[]
duplicates = [k for k,v in Counter(feat).items() if v>1]
feat = list(set(feat) - set(duplicates))
[k for k,v in Counter(feat).items() if v>1]

In [ ]:
# Function that trains a 5-group-fold lightgbm model.  Accepts parameter dictionary and feature list as parameters.
# Returns cohen kappa CV score, all 5 models, list of predictions on the reduce_test_tr test set.

def train_lgb(params, feat):
    n_fold = 5
    folds = GroupKFold(n_splits=n_fold)
    oof = np.zeros(y.shape[0])
    models = []
    coefs = [1.09122991, 1.75101234, 2.24616994]
    print(len(feat))
    for fold, (tr_idx, v_idx) in enumerate(folds.split(reduce_train, y, reduce_train['installation_id'])):
        print(f'Fold: {fold}\n{"*"*50}')
        X_train, y_train = reduce_train.iloc[tr_idx], y.iloc[tr_idx]
        X_valid, y_valid = reduce_train.iloc[v_idx], y.iloc[v_idx]

        lgb_train = lgb.Dataset(X_train[feat], y_train, categorical_feature=categoricals)
        lgb_valid = lgb.Dataset(X_valid[feat], y_valid, categorical_feature=categoricals)

        gbm_regress = lgb.train(params, lgb_train, verbose_eval=50, valid_sets=[lgb_train, lgb_valid], num_boost_round=5000,
                                early_stopping_rounds=50, categorical_feature=categoricals)
        oof[v_idx] = gbm_regress.predict(X_valid[feat], num_iteration=gbm_regress.best_iteration, random_state=28)
        models.append(gbm_regress)    
    y_pred = sum([model.predict(reduce_test_tr[feat], num_iteration=model.best_iteration) for model in models])/n_fold
    y_true = y_t
    opt2 = sp.optimize.minimize(fun=obj_func, x0=np.array([1.22232214, 1.73925866, 2.22506454]), args=(y_pred, y_true), method='nelder-mead')
    score = opt2.fun
    print(score)
    return score, models, y_pred

In [ ]:
# These features were sources of overfitting.  Drop from the model.

drop_cols = ['installation_session_clip_count', 'installation_session_act_count', 'installation_session_game_count']
feat = [x for x in feat if x not in drop_cols]

In [ ]:
%%time

# Run feature selection as described at the beginning of the notebook.  The same set of paramaters are used in each iteration.

steps = [6, 7] if run_long_version else [3, 2]

features_dict = {}
models_dict = {}
oof_vectors = {}
scores_final = []
params = {'n_estimators':2000,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'subsample': 0.75,
        'subsample_freq': 1,
        'learning_rate': 0.04,
        'feature_fraction': 0.9,
        'max_depth': 15,
        'lambda_l1': 1,  
        'lambda_l2': 1,
        }
n_samples = 12
baseline, _, _ = train_lgb(params, feat)
for i in range(steps[0]):
    feats = feat
    base = baseline
    for drops in range(steps[1]):
        score = 10
        tracking = 0
        while (score > base) & (tracking < n_samples):
            tracking += 1
            print(f'Iteration: {tracking}')
            poss = [x for x in feats if x not in ['session_title']]
            new_feats = random.sample(poss, k=len(feats)-int(len(feats)/10))
            new_feats.append('session_title')
            score, models, oof = train_lgb(params, new_feats)
        print(f'Score: {score}\nBase: {base}\nNumber of Features: {len(feats)}')
        if score < base:
            base = score
            feats = new_feats
            features_dict[f'{i}_iteration_feat'] = feats
            models_dict[f'{i}_iteration_models'] = models
            oof_vectors[f'{i}_iteration_oof'] = oof
    scores_final.append(score)

In [ ]:
# Print the 6 best scores.
print(scores_final)

## Ensembling

In [ ]:
# Function for use with sp.optimise, returns best linear coefficients.

def cappa_combinations(weights, x, y_true):
    y_pred = np.sum(weights * x, axis=1).values
    opt2 = sp.optimize.minimize(fun=obj_func, x0=np.array([1.22232214, 1.73925866, 2.22506454]), args=(y_pred, y_true), method='nelder-mead')
    score = opt2.fun
    return score

In [ ]:
# Score each subset of three predictions on the reduce_test_tr set.

import itertools
blend_scores = {}
t = pd.concat([pd.Series(x) for x in oof_vectors.values()], axis=1)
ls = [i for i in range(len(oof_vectors))]
for subset in itertools.combinations(ls, 3):
    subset = list(subset)
    opt2 = sp.optimize.minimize(fun=cappa_combinations, x0=np.array([0.33, 0.33, 0.33]), args=(t[subset], y_t), method='nelder-mead')
    score = opt2.fun
    print(score)
    blend_score = score
    weights = opt2.x
    print(opt2.x)
    y_pred = np.sum(weights * t[subset], axis=1).values
    opt2 = sp.optimize.minimize(fun=obj_func, x0=np.array([1.22232214, 1.73925866, 2.22506454]), args=(y_pred, y_t), method='nelder-mead')
    coefs = opt2.x
    blend_scores[f'{blend_score}'] = [subset, weights, coefs]

In [ ]:
# Print the minimum score, with the index of each model used.

comb = blend_scores[min(blend_scores)]
print(comb)

## Predictions

In [ ]:
# Create ensembled test predictions.

n_fold = 5
test_preds = pd.DataFrame()
for i in comb[0]:
    models = models_dict[f'{i}_iteration_models']
    feats = features_dict[f'{i}_iteration_feat']
    pred = sum([model.predict(reduce_test[feats], num_iteration=model.best_iteration) for model in models])/n_fold
    test_preds[f'{i}_preds'] = pred

## Submission

In [ ]:
# Create submission.  Final graph showing distribution of class values.

y_pred = np.sum(comb[1] * test_preds.values, axis=1)
y_pred = use_coefs(y_pred, comb[2], train_preds=False)
sample_submission = pd.read_csv(root + 'sample_submission.csv')
sample_submission['accuracy_group'] = y_pred.astype('int')
sample_submission.to_csv('submission.csv', index=False)
print(sample_submission['accuracy_group'].value_counts())
print(sample_submission)
sample_submission['accuracy_group'].hist(figsize=(12, 8))